In [47]:
import pandas as pd 
import os
import csv

In [48]:
# pulling in clean data 

Table2a_filepath = os.path.join("Clean Data\Table 2a.csv")
Table2b_filepath = os.path.join("Clean Data\Table 2b.csv")
Table3_filepath = os.path.join("Clean Data\Table 3.csv")

US_renewable = pd.read_csv(Table2a_filepath)
US_oil = pd.read_csv(Table2b_filepath)
USD_data = pd.read_csv(Table3_filepath)

US_oil.head()

,Date,Value (million kWh),Description
0,2005-01-01,177013.907,"Electricity Net Generation From Coal, All Sectors"
1,2005-02-01,155818.014,"Electricity Net Generation From Coal, All Sectors"
2,2005-03-01,163612.700,"Electricity Net Generation From Coal, All Sectors"
3,2005-04-01,143083.239,"Electricity Net Generation From Coal, All Sectors"
4,2005-05-01,153957.962,"Electricity Net Generation From Coal, All Sectors"


In [49]:
US_oil["Description"].unique()

array(['Electricity Net Generation From Coal, All Sectors',
       'Electricity Net Generation From Petroleum, All Sectors',
       'Electricity Net Generation From Natural Gas, All Sectors',
       'Electricity Net Generation From Other Gases, All Sectors',
       'Electricity Net Generation From Conventional Hydroelectric Power, All Sectors'],
      dtype=object)

In [50]:
#group by source types 

Coal = US_oil.loc[US_oil["Description"] == "Electricity Net Generation From Coal, All Sectors"]
Coal = Coal.rename(columns={"Value (million kWh)" : "Coal (mil kWh)"})
Coal = Coal.drop(columns=["Description"])

Petroleum = US_oil.loc[US_oil["Description"] == "Electricity Net Generation From Petroleum, All Sectors"]
Petroleum = Petroleum.rename(columns={"Value (million kWh)" : "Petroleum (mil kWh)"})
Petroleum = Petroleum.drop(columns=["Description"])

Natural_Gas = US_oil.loc[US_oil["Description"] == "Electricity Net Generation From Natural Gas, All Sectors"]
Natural_Gas = Natural_Gas.rename(columns={"Value (million kWh)" : "Natural_Gas (mil kWh)"})
Natural_Gas = Natural_Gas.drop(columns=["Description"])

Other_Gas = US_oil.loc[US_oil["Description"] == "Electricity Net Generation From Other Gases, All Sectors"]
Other_Gas = Other_Gas.rename(columns={"Value (million kWh)" : "Other_Gas (mil kWh)"})
Other_Gas = Other_Gas.drop(columns=["Description"])

Conv_Power = US_oil.loc[US_oil["Description"] == "Electricity Net Generation From Conventional Hydroelectric Power, All Sectors"]
Conv_Power = Conv_Power.rename(columns={"Value (million kWh)" : "Conventional Hydroelectric Power (mil kWh)"})
Conv_Power = Conv_Power.drop(columns=["Description"])

Coal.head()

,Date,Coal (mil kWh)
0,2005-01-01,177013.907
1,2005-02-01,155818.014
2,2005-03-01,163612.700
3,2005-04-01,143083.239
4,2005-05-01,153957.962


In [51]:
#combining data frames 

US_oil = Coal.merge(Petroleum, on="Date", how="left")
US_oil = US_oil.merge(Natural_Gas, on="Date", how="left")
US_oil = US_oil.merge(Other_Gas, on="Date", how="left")
US_oil = US_oil.merge(Conv_Power, on="Date", how="left")

US_oil = US_oil.set_index(["Date"])
US_oil.head()

,Coal (mil kWh),Petroleum (mil kWh),Natural_Gas (mil kWh),Other_Gas (mil kWh),Conventional Hydroelectric Power (mil kWh)
Date,,,,,
2005-01-01,177013.907,12190.178,51337.627,1126.191,24272.165
2005-02-01,155818.014,7314.510,44912.617,1076.051,21606.774
2005-03-01,163612.700,8351.900,51896.973,1214.487,22936.068
2005-04-01,143083.239,6952.153,52016.286,1121.459,23058.359
2005-05-01,153957.962,6713.918,54826.004,1191.303,27278.511


In [52]:
#changing data type so we can get the total sum
US_oil = US_oil.astype("float64")

In [53]:
#Getting total electricity production

US_oil["Total Oil (mil kWh)"] = US_oil.sum(axis=1)
US_oil.head()


,Coal (mil kWh),Petroleum (mil kWh),Natural_Gas (mil kWh),Other_Gas (mil kWh),Conventional Hydroelectric Power (mil kWh),Total Oil (mil kWh)
Date,,,,,,
2005-01-01,177013.907,12190.178,51337.627,1126.191,24272.165,265940.068
2005-02-01,155818.014,7314.510,44912.617,1076.051,21606.774,230727.966
2005-03-01,163612.700,8351.900,51896.973,1214.487,22936.068,248012.128
2005-04-01,143083.239,6952.153,52016.286,1121.459,23058.359,226231.496
2005-05-01,153957.962,6713.918,54826.004,1191.303,27278.511,243967.698


In [54]:
US_renewable["Description"].unique()

array(['Electricity Net Generation From Hydroelectric Pumped Storage, All Sectors',
       'Electricity Net Generation From Wood, All Sectors',
       'Electricity Net Generation From Waste, All Sectors',
       'Electricity Net Generation From Geothermal, All Sectors',
       'Electricity Net Generation From Solar, All Sectors',
       'Electricity Net Generation From Wind, All Sectors'], dtype=object)

In [55]:
#group by source types 
#hydro electric pump is just a storage unit

wood = US_renewable.loc[US_renewable["Description"] == "Electricity Net Generation From Wood, All Sectors"]
wood = wood.rename(columns={"Value (million kWh)" : "Wood (mil kWh)"})
wood = wood.drop(columns=["Description"])

waste = US_renewable.loc[US_renewable["Description"] == "Electricity Net Generation From Waste, All Sectors"]
waste = waste.rename(columns={"Value (million kWh)" : "Waste (mil kWh)"})
waste = waste.drop(columns=["Description"])

geothermal = US_renewable.loc[US_renewable["Description"] == "Electricity Net Generation From Geothermal, All Sectors"]
geothermal = geothermal.rename(columns={"Value (million kWh)" : "Geothermal (mil kWh)"})
geothermal = geothermal.drop(columns=["Description"])

solar = US_renewable.loc[US_renewable["Description"] == "Electricity Net Generation From Solar, All Sectors"]
solar = solar.rename(columns={"Value (million kWh)" : "Solar (mil kWh)"})
solar = solar.drop(columns=["Description"])

wind = US_renewable.loc[US_renewable["Description"] == "Electricity Net Generation From Wind, All Sectors"]
wind = wind.rename(columns={"Value (million kWh)" : "Wind (mil kWh)"})
wind = wind.drop(columns=["Description"])

wind.head()

,Date,Wind (mil kWh)
900,2005-01-01,1131.569
901,2005-02-01,966.482
902,2005-03-01,1560.626
903,2005-04-01,1697.504
904,2005-05-01,1746.430


In [56]:
#combining data frames 

US_renewable = wood.merge(waste, on="Date", how="left")
US_renewable = US_renewable.merge(geothermal, on="Date", how="left")
US_renewable = US_renewable.merge(solar, on="Date", how="left")
US_renewable = US_renewable.merge(wind, on="Date", how="left")

US_renewable = US_renewable.set_index(["Date"])
US_renewable.head()

,Wood (mil kWh),Waste (mil kWh),Geothermal (mil kWh),Solar (mil kWh),Wind (mil kWh)
Date,,,,,
2005-01-01,3333.076,1278.347,1252.144,8.668,1131.569
2005-02-01,3055.295,1123.407,1063.014,13.082,966.482
2005-03-01,3287.717,1277.929,1204.252,38.157,1560.626
2005-04-01,3035.547,1222.542,1187.134,58.263,1697.504
2005-05-01,3100.697,1351.152,1264.308,81.153,1746.430


In [57]:
#changing data type so we can get the total sum
US_renewable = US_renewable.astype("float64")

In [58]:
#Getting total electricity production

US_renewable["Total Renewable (mil kWh)"] = US_renewable.sum(axis=1)
US_renewable.head()

,Wood (mil kWh),Waste (mil kWh),Geothermal (mil kWh),Solar (mil kWh),Wind (mil kWh),Total Renewable (mil kWh)
Date,,,,,,
2005-01-01,3333.076,1278.347,1252.144,8.668,1131.569,7003.804
2005-02-01,3055.295,1123.407,1063.014,13.082,966.482,6221.280
2005-03-01,3287.717,1277.929,1204.252,38.157,1560.626,7368.681
2005-04-01,3035.547,1222.542,1187.134,58.263,1697.504,7200.990
2005-05-01,3100.697,1351.152,1264.308,81.153,1746.430,7543.740


In [59]:
#Set index as data 

USD_data = USD_data.set_index(["Date"], drop=True)
USD_data.head()

,Value
Date,
2005-01-01,95.450
2005-02-01,95.698
2005-03-01,95.307
2005-04-01,96.234
2005-05-01,96.470


In [60]:
#rename column name 

USD_data = USD_data.rename( columns={ "Value" : "USD Index"})
USD_data.head()

,USD Index
Date,
2005-01-01,95.450
2005-02-01,95.698
2005-03-01,95.307
2005-04-01,96.234
2005-05-01,96.470


In [68]:
#preparing data into organized sets for plotting purposes

#investigation of electricity production (oil and renewable sources) vs. time 
all_elec = US_renewable.merge(US_oil, on="Date")
elec_production = all_elec[ ["Total Oil (mil kWh)", "Total Renewable (mil kWh)"]]
elec_production.to_csv(r'Transformed Data\Question 5.csv')

#investigation of electricity production (renewable and its sources) vs. time 
US_renewable.to_csv(r'Transformed Data\Question 5B.csv')

#investigation of electricity production (oil and its sources) vs. time 
US_oil.to_csv(r'Transformed Data\Question 5C.csv')

#investigation of electricity production by renewable sources vs. USD index 
USD_impact = US_renewable.merge(USD_data, on="Date")
USD_impact.to_csv(r'Transformed Data\Question 6.csv')
